In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lastman0800/cyberattacks-detection")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cyberattacks-detection


In [5]:
import pandas as pd
import seaborn as sns
pathDF = "/kaggle/input/cyberattacks-detection/Cyberattacks Detection.csv"

cyberData = pd.read_csv(pathDF)
cyberData


/tmp/ipython-input-653690535.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  cyberData = pd.read_csv(pathDF)


,Attack ID,Source IP,Destination IP,Source Country,Destination Country,Protocol,Source Port,Destination Port,Attack Type,Payload Size (bytes),Detection Label,Confidence Score,ML Model,Affected System,Port Type,Timestamp
0,1.0,68.156.227.194,194.91.121.112,India,USA,ICMP,16824.0,53410.0,Brute Force,3936.0,Detected,0.600507,K-Nearest Neighbors,Cloud Storage,Other,NaN
1,2.0,249.221.229.41,3.207.120.235,Germany,USA,TCP,23977.0,31745.0,Advanced Persistent Threat (APT),2870.0,Detected,0.524150,K-Nearest Neighbors,Network Router,Other,NaN
2,4.0,154.55.209.171,132.115.33.82,USA,China,UDP,36556.0,38880.0,Cross-Site Scripting (XSS) Attack,3833.0,Not Detected,0.238944,Neural Network,Workstation,Other,NaN
3,5.0,106.61.204.27,40.61.49.130,France,France,UDP,14724.0,21669.0,Insider Threat,4301.0,Not Detected,0.122334,Neural Network,Workstation,Other,NaN
4,6.0,144.190.250.125,34.68.66.192,South Korea,Germany,TCP,44693.0,6474.0,Watering Hole Attack,4569.0,Detected,0.970895,Neural Network,Application Server,Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996.0,249.190.160.222,92.174.237.234,China,UK,ICMP,16359.0,53207.0,Cross-Site Scripting (XSS) Attack,2152.0,Detected,0.765800,Support Vector Machine,Email Server,Other,6/16/2024 10:34
99996,99997.0,86.76.247.65,49.174.181.21,India,USA,ICMP,21591.0,44627.0,Simple Network Management Protocol (SNMP) Attack,598.0,Detected,0.864282,K-Nearest Neighbors,Cloud Storage,Other,6/10/2024 3:51
99997,99998.0,3.150.34.188,39.26.219.220,USA,Brazil,ICMP,5895.0,26251.0,Brute Force,1570.0,Detected,0.723099,K-Nearest Neighbors,Web Server,Other,7/8/2024 19:54
99998,99999.0,144.151.163.243,215.162.102.106,South Korea,China,ICMP,45336.0,46630.0,Drive-by Download,2987.0,Detected,0.564727,Random Forest,Network Router,Other,11/11/2024 20:14


In [6]:
cyberData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Attack ID             99940 non-null  float64
 1   Source IP             99950 non-null  object 
 2   Destination IP        99945 non-null  object 
 3   Source Country        99950 non-null  object 
 4   Destination Country   99947 non-null  object 
 5   Protocol              99941 non-null  object 
 6   Source Port           99949 non-null  float64
 7   Destination Port      99946 non-null  float64
 8   Attack Type           99941 non-null  object 
 9   Payload Size (bytes)  99947 non-null  float64
 10  Detection Label       99933 non-null  object 
 11  Confidence Score      99946 non-null  float64
 12  ML Model              99939 non-null  object 
 13  Affected System       99954 non-null  object 
 14  Port Type             99939 non-null  object 
 15  Timestamp         

In [7]:
cyberData.corr(numeric_only=True)

,Attack ID,Source Port,Destination Port,Payload Size (bytes),Confidence Score
Attack ID,1.000000,-0.000241,0.001000,0.000159,0.002497
Source Port,-0.000241,1.000000,-0.003562,0.001310,-0.005835
Destination Port,0.001000,-0.003562,1.000000,0.000064,0.000561
Payload Size (bytes),0.000159,0.001310,0.000064,1.000000,0.003224
Confidence Score,0.002497,-0.005835,0.000561,0.003224,1.000000


In [8]:
for col in cyberData.columns:
    print('\t%s: %d' % (col,cyberData[col].isna().sum()))

	Attack ID: 60
	Source IP: 50
	Destination IP: 55
	Source Country: 50
	Destination Country: 53
	Protocol: 59
	Source Port: 51
	Destination Port: 54
	Attack Type: 59
	Payload Size (bytes): 53
	Detection Label: 67
	Confidence Score: 54
	ML Model: 61
	Affected System: 46
	Port Type: 61
	Timestamp: 90174


In [9]:
cyberDataCleaned = cyberData.dropna()
print(f"Size Before Dropping NaN: {cyberData.shape[0]}")
print(f"\tSize After Dropping NaN: {cyberDataCleaned.shape[0]}")


Size Before Dropping NaN: 100000
	Size After Dropping NaN: 9826


In [10]:
for col in cyberDataCleaned.columns:
    print('\t%s: %d' % (col,cyberDataCleaned[col].isna().sum()))

	Attack ID: 0
	Source IP: 0
	Destination IP: 0
	Source Country: 0
	Destination Country: 0
	Protocol: 0
	Source Port: 0
	Destination Port: 0
	Attack Type: 0
	Payload Size (bytes): 0
	Detection Label: 0
	Confidence Score: 0
	ML Model: 0
	Affected System: 0
	Port Type: 0
	Timestamp: 0


In [11]:
dupes = cyberData.duplicated()
print(dupes.sum())

0


In [12]:
print(cyberDataCleaned.dtypes)

Attack ID               float64
Source IP                object
Destination IP           object
Source Country           object
Destination Country      object
Protocol                 object
Source Port             float64
Destination Port        float64
Attack Type              object
Payload Size (bytes)    float64
Detection Label          object
Confidence Score        float64
ML Model                 object
Affected System          object
Port Type                object
Timestamp                object
dtype: object


In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA

nominal_categorical_features = ['Source Country', 'Destination Country','Detection Label','Affected System', 'Port Type','Protocol', 'Source IP', 'Destination IP', 'Confidence Score', 'ML Model']
numerical_features = ['Source Port','Destination Port','Payload Size (bytes)']

onehot_pipeline = Pipeline([
    ('onehot_encoder', OneHotEncoder(handle_unknown= 'ignore', sparse_output= False))
])
numeric_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
preproccesor = ColumnTransformer([
    ('onehot', onehot_pipeline, nominal_categorical_features),
    ('num', numeric_pipeline, numerical_features),
])

full_pipeline = Pipeline([
    ('preproccesor', preproccesor),
    ('select_features', SelectKBest(score_func=f_classif, k = 20))
    ('pca', PCA(n_components= .95)),
])


In [14]:
cyberDataCleaned['Attack Type'].value_counts()

,count
Attack Type,
Phishing,539
Brute Force,529
Ping of Death,523
File Transfer Protocol (FTP) Attack,517
Malware,504
Man-in-the-Middle (MITM),502
Simple Network Management Protocol (SNMP) Attack,497
Cross-Site Scripting (XSS) Attack,496
DNS Tunneling,489


In [ ]:
from sklearn.model_selection import train_test_split
X = cyberDataCleaned.drop('Attack Type', axis = 1)
y = cyberDataCleaned['Attack Type']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2, random_state=42, stratify=y)

X_train_encoded = full_pipeline.fit_transform(X_train, y_train)
X_test_encoded = full_pipeline.transform(X_test)


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [ ]:
print("TARGET VARIABLE ANALYSIS")
print("Attack Type value counts:")
attack_counts = cyberDataCleaned['Attack Type'].value_counts()
print(attack_counts)
print(f"\nNumber of unique attack types: {cyberDataCleaned['Attack Type'].nunique()}")



In [ ]:
print('Class Imbalance Analysis')
min_class_size = attack_counts.min()
max_class_size = attack_counts.max()
imbalance_ratio = max_class_size/min_class_size
print(f"Imbalance Ratio: {imbalance_ratio}")


In [ ]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
param_grid = {
   'kernel': ['rbf', 'linear'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced']
}
grid_search = GridSearchCV(SVC(), param_grid, cv = 3)
grid_search.fit(X_train_encoded, y_train_encoded)
print(f"Best SVC model is: {grid_search.best_estimator_}")

In [ ]:

param_grid_sgd = {
    'penalty': ['l1','l2'],
    'loss': ['log_loss','hinge']
}
grid_search_sgd = GridSearchCV(SGDClassifier(random_state= 42, early_stopping= True, eta0= .01, class_weight='balanced'), param_grid_sgd, cv = 5)
grid_search_sgd.fit(X_train_encoded,y_train_encoded)
print(f"Best SGD model: {grid_search_sgd.best_estimator_}")


In [ ]:
param_grid_ridge = {
    'alpha': [.01,.1,1],
    'solver': ["auto", "cholesky"]
}
grid_search_ridge = GridSearchCV(RidgeClassifier(),param_grid_ridge, cv = 5)
grid_search_ridge.fit(X_train_encoded, y_train_encoded)
print(f"Best Ridge Model: {grid_search_ridge.best_estimator_}")

In [ ]:
param_grid_dt = {
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced']
}
grid_search_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv = 5)
grid_search_dt.fit(X_train_encoded, y_train_encoded)
print(f"Best DT Model: {grid_search_dt.best_estimator_}")

In [ ]:
print(f"Best SVC score: {grid_search.best_score_:.4f}")
print(f"Best SGD score: {grid_search_sgd.best_score_:.4f}")
print(f"Best Ridge score: {grid_search_ridge.best_score_:.4f}")
print(f"Best DT score: {grid_search_dt.best_score_:4f}")

# Get the best models
best_svc = grid_search.best_estimator_
best_sgd = grid_search_sgd.best_estimator_
best_ridge = grid_search_ridge.best_estimator_
best_dt = grid_search_dt.best_estimator_

In [ ]:
y_pred_cvs = best_svc.predict(X_test_encoded)


In [ ]:
from sklearn.metrics import accuracy_score
AccScore = (y_test, y_pred_cvs)
print(AccScore)

In [ ]:
svc = SVC(kernel = 'linear', coef0=3, C = 1, class_weight='balanced')
svc.fit(X_train_encoded, y_train_encoded)
y_pred_svc = svc.predict(X_test_encoded)


In [ ]:
cr_svc = classification_report(y_pred_svc, y_test)
print(cr_svc)

In [ ]:
import tensorflow as tf
from tensorflow import keras

nn = keras.Sequential([
    keras.layers.Dense(32, activation="relu", kernel_regularizer=keras.regularizers.l1(0.1)),
    keras.layers.Dense(20, activation="softmax")
])
nn.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
nn.summary()


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels as integers
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

nn.fit(X_train_encoded, y_train_encoded, epochs=50, verbose=1)

In [ ]:
# 1. Simple preprocessing - just encode categoricals and scale numerics
simple_preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_categorical_features),
    ('num', StandardScaler(), numerical_features)
])

# 2. Prepare raw data without PCA or feature selection
X_raw = cyberDataCleaned.drop(['Attack Type'], axis=1)
y_raw = cyberDataCleaned['Attack Type']

# 3. Split the data
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# 4. Apply minimal preprocessing
X_train_simple = simple_preprocessor.fit_transform(X_train_raw)
X_test_simple = simple_preprocessor.transform(X_test_raw)

# 5. Encode target
y_train_simple = le.fit_transform(y_train_raw)
y_test_simple = le.transform(y_test_raw)

# 6. Train neural network on minimally processed data
nn_simple = keras.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(20, activation="softmax")
])

nn_simple.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# 7. Fit and evaluate
history = nn_simple.fit(
    X_train_simple,
    y_train_simple,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# 8. Print evaluation metrics
test_loss, test_accuracy = nn_simple.evaluate(X_test_simple, y_test_simple)
print(f"\nTest accuracy with minimal preprocessing: {test_accuracy:.4f}")